# Variance Reduction - Weight Windows

## Iteratively creating and utilizing a wight window to accelerate deep shielding simulations

This example simulates a sphere of material with a neutron source in the center.This example implements the MAGIC method of weight window generation on each simulation run.

In this tutorial we shall focus on generating a weight window to accelerate the simulation of particles through a shield and improving the weight window with each iteration.

Weight Windows are found using the MAGIC method and used to accelerate the simulation.

The variance reduction method used for this simulation is well documented in the OpenMC documentation
https://docs.openmc.org/en/stable/methods/neutron_physics.html

The MAGIC method is well described in the original publication
https://scientific-publications.ukaea.uk/wp-content/uploads/Published/INTERN1.pdf


First we import ```openmc``` including ```openmc.lib``` and other packages needed for the example

In [ ]:
import openmc
import openmc.lib  # this example makes use of openmc lib to run the simulations

import numpy as np

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm  # used for plotting log scale graphs

We create a couple of materials for the simulation

In [ ]:
mat_water = openmc.Material()
mat_water.add_element("H", 1)
mat_water.add_element("O", 2)
mat_water.set_density("g/cm3", 1.0)

my_materials = openmc.Materials([mat_water])

Now we define and plot the spherical geometry.

In [ ]:
# outer surface at 500 cm
outer_surface = openmc.model.RectangularParallelepiped(-300, 300, -300, 300, -300, 300, boundary_type="vacuum")

# A single region below the surface
region_1 = -outer_surface

# A single cell full of water
cell_1 = openmc.Cell(region=region_1)
cell_1.fill = mat_water

my_geometry = openmc.Geometry([cell_1])

Now we plot the geometry and color by materials.

In [ ]:
my_geometry.root_universe.plot(basis='xy', color_by='material') 
plt.savefig('geometry_top_down_view.png', bbox_inches="tight")

Next we create a point source, this also uses the same geometry parameters to place in the center of the room regardless of the values of the parameters.

In [ ]:
# location of the point source
space = openmc.stats.Point((0, 0, 0))
angle = openmc.stats.Isotropic()

# all (100%) of source particles are 14MeV energy
energy = openmc.stats.Discrete([14e6], [1.0])

source = openmc.Source(space=space, angle=angle, energy=energy)
source.particle = "neutron"

Next we create a mesh that encompasses the entire geometry and scores neutron flux

In [ ]:
mesh = openmc.RegularMesh().from_domain(domain=my_geometry)
print(mesh)
# mesh.r_grid = np.linspace(0, outer_surface.r, 100)

mesh_filter = openmc.MeshFilter(mesh)

flux_tally = openmc.Tally(name="flux tally")
flux_tally.filters = [mesh_filter]
flux_tally.scores = ["flux"]
flux_tally.id = 55  # we set the ID number here as we need to access it during the openmc lib run

# adds the mesh tally to the model
my_tallies = openmc.Tallies()
my_tallies.append(flux_tally)

tallies = openmc.Tallies([flux_tally])

Creates the simulation settings

In [ ]:
my_settings = openmc.Settings()
my_settings.run_mode = "fixed source"
my_settings.source = source
my_settings.particles = 120
my_settings.batches = 10

# no need to write the tallies.out file which saves space and time when large meshes are used
my_settings.output = {'tallies': False}

Creates and export the model

In [ ]:
model = openmc.Model(my_geometry, my_materials, my_settings, my_tallies)

# deletes old input and output files
!rm *.xml
!rm *.h5

model.export_to_xml()  # this is necessary as openmc.lib loads up the model.xml file

Now we want to plot the results of the simulation. We want to do this twice to compare the results so I've written this up as a function that we can call.

In [ ]:
def plot_mesh_tally_and_weight_window(statepoint_filename, weight_window_filename, image_filename):
    
    with openmc.StatePoint(statepoint_filename) as sp:
        flux_tally = sp.get_tally(name="flux tally")

    tally_mesh = flux_tally.find_filter(openmc.MeshFilter).mesh
    tally_mesh_extent = tally_mesh.bounding_box.extent['xy']

    # get a slice of mean values on the xy basis mid z axis
    flux_mean = flux_tally.get_reshaped_data(value='mean', expand_dims=True).squeeze()[:,:,int(mesh.dimension[2]/2)]
    plt.subplot(1, 3, 1)
    # create a plot of the mean flux values
    plt.imshow(
        flux_mean.T,
        origin="lower",
        extent=tally_mesh_extent,
        norm=LogNorm(),
    )
    plt.title("Flux Mean")


    plt.subplot(1, 3, 2)
    # get a slice of std dev values on the xy basis mid z axis
    flux_std_dev = flux_tally.get_reshaped_data(value='std_dev', expand_dims=True).squeeze()[:,:,int(mesh.dimension[1]/2)]
    # create a plot of the flux relative error
    plt.imshow(
        flux_std_dev.T,
        origin="lower",
        extent=tally_mesh_extent,
        norm=LogNorm(),
    )
    plt.title("Flux Std. Dev.")

    wws=openmc.hdf5_to_wws(weight_window_filename)
    ww = wws[0]  # get the one and only mesh tally
    ww_mesh = ww.mesh  # get the mesh that the weight window is mapped on
    ww_mesh_extent = ww_mesh.bounding_box.extent['xy']
    reshaped_ww_vals = ww.lower_ww_bounds.reshape(mesh.dimension)
    print('reshaped_ww_vals.shape', reshaped_ww_vals.shape)
    # slice on XZ basis, midplane Y axis
    slice_of_ww = reshaped_ww_vals[:,:,int(mesh.dimension[1]/2)]
    plt.subplot(1, 3, 3)
    plt.imshow(
        slice_of_ww.T,
        origin="lower",
        extent=ww_mesh_extent,
        norm=LogNorm(),
    )
    plt.title("Weight Window lower bound")

    plt.savefig(image_filename, bbox_inches="tight")

Now we make use of openmc.lib to control the simulation. Documentation on openmc.lib is here
https://docs.openmc.org/en/stable/pythonapi/capi.html

We run 5 iterations with each iteration improving the weight window.

In [ ]:
with openmc.lib.run_in_memory():

    # loads up a live pointer to the tally with id=55, at this stage the tally is empty
    tally = openmc.lib.tallies[55]

    # makes weight windows from the tally, at this stage the values are empty
    wws = openmc.lib.WeightWindows.from_tally(tally)

    # 5 iterations of weight window improvements
    for i in range(5):

        # run the simulation
        openmc.lib.run()

        # improves the weight window with the latest tally results
        wws.update_magic(tally)

        # we write out the weight window map for plotting later
        openmc.lib.export_weight_windows(filename=f'weight_windows{i}.h5')
        # we write out the statepoint so that the tally can be plotted later
        openmc.lib.statepoint_write(filename=f'statepoint_simulation_{i}.h5')

        # turns on the weight windows to ensure they are used
        openmc.lib.settings.weight_windows_on = True

        # creates a plot of the flux, std_dev and weight window
        plot_mesh_tally_and_weight_window(
            f'statepoint_simulation_{i}.h5',
            f'weight_windows{i}.h5',
            f'plot_{i}.png'
        )

The iterative improvment of the flux / standard deviation / weight windows with each simulation run can be seen when we plot all the simulation results one after each other.

In [ ]:
from PIL import Image
images = [Image.open(x) for x in [f'plot_{c}.png' for c in range(5)]]
widths, heights = zip(*(i.size for i in images))

total_height = sum(heights)
max_width = max(widths)

new_im = Image.new('RGB', (max_width, total_height))
y_offset = 0
for im in images:
  new_im.paste(im, (0,y_offset))
  y_offset += im.size[1]
new_im.save('flux_std-dev_ww_for_all_simulations_reset.png')
new_im

Learning outcome

Weight windows can be incrementally improved by running subsequent simulations.

Running lots of small simulations where the weight window can incrementally improve the wieght window and can yields better results that a big single simulation to generate weight windows and a single big simulation to make use of the weight windows.

Doing this iteration with openmc.lib means we don't need to reload the nuclear data between simulations which saves time.

Additionally we have access to openmc.lib methods which are nessecary for updating the weight window with the MAGIC method and exporting the weight window to a h5 file.
